In [332]:
import requests
import json
from bs4 import BeautifulSoup
import re
from pandas import Timestamp as TS
import numpy as np

# download database of tasks

In [99]:
s = requests.Session()

In [343]:
def send_request(session, pageNumber, pageSize=10, numRetries =5):
    '''
    Each page has pageSize tasks
    '''

    headers = {
        #"Accept": "application/json, text/plain, */*",
        #"Accept-Encoding":  "gzip, deflate",
        #"Accept-Language":  "en-US,en;q=0.9",
        #"Connection":"keep-alive",
        "Content-Type" : "application/json;charset=UTF-8",
        #"Cookie": "__ddg1=o6KApivyOlKgboqolkZG; sessionId=321ec231-18ab-8f70-49f0-870baefdb40a",
        #"Host":"os.fipi.ru",
        #"Origin" :"http://os.fipi.ru",
        #"Referer": "http://os.fipi.ru/tasks/1/a",
        "sessionId": "321ec231-18ab-8f70-49f0-870baefdb40a",
        #"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"
    }
    body = {
        'subjectId': '1', #Russian
        "pageNumber": pageNumber,
        "pageSize": pageSize,
        "answerStatus": 0,
        "docId" : "",
        "extId": "",
        "favorites": 0,
        "fipiCode": "",
        "id": "",
        "isAdmin": False, #CAN SET THIS TO TRUE AND IT WORKS (I get more tasks)
        "isPublished": False,
        "levelIds": [],
        "loadDates": [],
        "published": 0,
        "themeIds": [],
        "themeSectionIds": [],
        "typeIds": []

    #     "levelIds": [],
    #     "themeIds": [],
    #     "typeIds": [],
    #     "id": "",

    }
    data = json.dumps(body)
    #data = r'{"subjectId":"1","levelIds":[],"themeIds":[],"typeIds":[],"id":"","favorites":0,"answerStatus":0,"themeSectionIds":[],"published":0,"extId":"","fipiCode":"","docId":"","isAdmin":false,"loadDates":[],"isPublished":false,"pageSize":5,"pageNumber":"2"}'
    loaded = False
    for i in range(numRetries):
        try:
            r = session.post("http://os.fipi.ru/api/tasks", data = data,
                  headers = headers)
            loaded = True
        except ConnectionResetError:
            print('retrying')
            pass
    assert loaded
    assert r.status_code == 200
    return r.text


In [345]:
try:
    r = s.post("http://os.fipi.ru/api/tasks", data = data,
                  headers = headers)
except ConnectionResetError:
    print('failed')

In [349]:
save_file = "data.json"
pageSize= 10


with open(save_file, "w") as f:
    task_count = json.loads(send_request(s, pageNumber = 1, pageSize = 1))['taskCount']
    tasks_loaded = 0
    cur_page = 1
    while tasks_loaded < task_count:
        resp = send_request(s, pageNumber = cur_page, pageSize = pageSize)
        resp = json.loads(resp)
        assert task_count == resp['taskCount']
        resp['my_timestamp'] = str(TS.now())
        json.dump(resp, f)
        f.write("\n")
        cur_page +=1
        tasks_loaded += pageSize
        print(f"loaded {tasks_loaded} out of {task_count}")

loaded 10 out of 1799
loaded 20 out of 1799
loaded 30 out of 1799
loaded 40 out of 1799
loaded 50 out of 1799
loaded 60 out of 1799
loaded 70 out of 1799
loaded 80 out of 1799
loaded 90 out of 1799
loaded 100 out of 1799
loaded 110 out of 1799
loaded 120 out of 1799
loaded 130 out of 1799
loaded 140 out of 1799
loaded 150 out of 1799
loaded 160 out of 1799
loaded 170 out of 1799
loaded 180 out of 1799
loaded 190 out of 1799
loaded 200 out of 1799
loaded 210 out of 1799
loaded 220 out of 1799
loaded 230 out of 1799
loaded 240 out of 1799
loaded 250 out of 1799
loaded 260 out of 1799
loaded 270 out of 1799
loaded 280 out of 1799
loaded 290 out of 1799
loaded 300 out of 1799
loaded 310 out of 1799
loaded 320 out of 1799
loaded 330 out of 1799
loaded 340 out of 1799
loaded 350 out of 1799
loaded 360 out of 1799
loaded 370 out of 1799
loaded 380 out of 1799
loaded 390 out of 1799
loaded 400 out of 1799
loaded 410 out of 1799
loaded 420 out of 1799
loaded 430 out of 1799
loaded 440 out of 17

# parse tasks

In [299]:
task = parse_task(tasks[6])

task


Прочитайте текст и выполните задания.
******************************************
(1)… (2)Но бывало и по-другому. (3)В XVI веке испанский капитан Себастьян Кабот, плывя вверх по течению реки в Южной Америке, был поражён количеством серебра, которое было у местных индейцев, живших по берегам реки, и решил назвать её Ла-Платой, то есть серебряной  (по-испански «плата»   –   серебро). (4)… впоследствии произошло и название всей страны. (5)Однако в начале XIX века владычество Испании кончилось, и, чтобы не вспоминать об этом печальном периоде, жители страны латинизировали её название. (6)Так на географических картах появилось слово «Аргентина».
******************************************

******************************************
correct: 2

In [300]:
for i, task in enumerate(tasks):
    print(i)
    print(parse_task(task))

0

Внимательно прочитайте текст задания и выберите верный ответ из списка
******************************************
В каком слове верно выделена буква, обозначающая ударный гласный звук?
******************************************
1) кухОнный
2) нЕдуг
3) перелИла
4) звонИт
******************************************
correct: 4

1

Внимательно прочитайте текст задания и выберите верный ответ из списка
******************************************
В каком варианте ответа выделенное слово употреблено неверно?
******************************************
1) Вратарь хоккейной команды НАДЕВАЕТ экипировку, защищающую его от ударов шайбы.
2) Летняя олимпиада стала самым ВПЕЧАТЛИТЕЛЬНЫМ событием минувшего года для многих миллионов людей.
3) Если голос у исполнителя негромкий, а музыкальное сопровождение ЗВУЧНОЕ, артисту приходится постоянно напрягать связки.
4) Передел мирового энергетического рынка ВСТУПИЛ в новую фазу.
******************************************
correct: 2

2

Внимательно прочитайте

In [264]:
parse_task(tasks[6])


Прочитайте текст и выполните задания.
******************************************
(1)… (2)Но бывало и по-другому. (3)В XVI веке испанский капитан Себастьян Кабот, плывя вверх по течению реки в Южной Америке, был поражён количеством серебра, которое было у местных индейцев, живших по берегам реки, и решил назвать её Ла-Платой, то есть серебряной  (по-испански «плата»   – MathType@MTEF@5@5@+=feaaguart1ev2aqatCvAUfeBSjuyZL2yd9gzLbvyNv2CaerbuLwBLnhiov2DGi1BTfMBaeXatLxBI9gBaerbd9wDYLwzYbItLDharqqtubsr4rNCHbGeaGqipD0dg9qqqrpepC0xbbL8F4rqqrFfpeea0xe9Lq=Jc9vqaqpepm0xbba9pwe9Q8fs0=yqaqpepae9pg0FirpepeKkFr0xfr=xfr=xb9adbiqaaeaacaGaaiaabeqaamaabaabaaGcbaacbaqcLbCaqaaaaaaaaaWdbiaa=nbiaaa@3781@   серебро). (4)… впоследствии произошло и название всей страны. (5)Однако в начале XIX века владычество Испании кончилось, и, чтобы не вспоминать об этом печальном периоде, жители страны латинизировали её название. (6)Так на географических картах появилось слово «Аргентина».
*********************************

In [263]:
parse_task(tasks[7])


Задание №1. Внимательно прочитайте текст задания и выберите верный ответ из списка
******************************************
Какое из приведённых ниже предложений должно быть первым в этом тексте?
******************************************
1) Взгляните на таблицу Менделеева   – MathType@MTEF@5@5@+=feaaguart1ev2aqatCvAUfeBSjuyZL2yd9gzLbvyNv2CaerbuLwBLnhiov2DGi1BTfMBaeXatLxBI9gBaerbd9wDYLwzYbItLDharqqtubsr4rNCHbGeaGqipD0dg9qqqrpepC0xbbL8F4rqqrFfpeea0xe9Lq=Jc9vqaqpepm0xbba9pwe9Q8fs0=yqaqpepae9pg0FirpepeKkFr0xfr=xfr=xb9adbiqaaeaacaGaaiaabeqaamaabaabaaGcbaacbaqcLbCaqaaaaaaaaaWdbiaa=nbiaaa@3781@   вам охотно подтвердят это германий и франций, европий и америций.
2) Географическая карта не раз служила подсказкой при выборе имени для вновь открытых химических элементов.
3) Северная и Южная Америка составляют два самостоятельных материка и сильно отличаются по природным условиям.
4) Большинство населения Южной Америки говорит на испанском языке.
******************************************
corr

In [226]:
task= tasks[1]
parsed = BeautifulSoup(task['html'])
parsed.find("p", attrs = {"class":lambda x: x.lower()=="basis"})


<p class="Basis">В каком варианте ответа выделенное слово употреблено <b>неверно</b>?</p>

'Внимательно прочитайте текст задания и выберите верный ответ из списка'

In [241]:
BeautifulSoup(tasks[6]['html'])

<!DOCTYPE html>
<html test=""><head>
    <title>Нулевая страница</title>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
    <meta charset="utf-8"/>
    <link href="/Content/KimTemplate/kimTmp.css" rel="stylesheet"/>
    <link href="/Content/KimTemplate/kimTmpCheckBox.css" rel="stylesheet"/>
    <script src="/Scripts/es6-shim/es6-shim.min.js"></script>
    <link href="/Content/task-common.css" rel="stylesheet"/>
    <script src="/scripts/task-common.bundle.js"></script>
    <script src="/scripts/task-helper.js"></script>
    <script src="/scripts/shuffleHelper.js"></script>
    <script src="/scripts/fipi-tools.js?rev=2"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=MML_HTMLorMML"></script>
    <script type="text/x-mathjax-config">
        MathJax.Hub.Config({messageStyle: "none"});
    </script>
</head>
<body class="custom-scroll-no inner-answers test noselect">
    <input id="TaskId" type="hidden" value="1569

In [240]:
BeautifulSoup(tasks[5]['html'])

<!DOCTYPE html>
<html test=""><head>
    <title tasktype="RadioButton">Внимательно прочитайте текст задания и выберите верный ответ из списка</title>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
    <meta charset="utf-8"/>
    <link href="/Content/KimTemplate/kimTmp.css" rel="stylesheet"/>
    <link href="/Content/KimTemplate/kimTmpCheckBox.css" rel="stylesheet"/>
    <script src="/Scripts/es6-shim/es6-shim.min.js"></script>
    <link href="/Content/task-common.css" rel="stylesheet"/>
    <script src="/scripts/task-common.bundle.js"></script>
    <script src="/scripts/task-helper.js"></script>
    <script src="/scripts/shuffleHelper.js"></script>
    <script src="/scripts/RadioButtonTest.js"></script>
    <script src="/scripts/fipi-tools.js?rev=2"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=MML_HTMLorMML"></script>
    <script type="text/x-mathjax-config">
        MathJax.Hub.Config({messageStyle: "none"}

In [237]:
for task in tasks:
    res = BeautifulSoup(task['html']).find("div", attrs = {"name": "text"}).text
    print(res)


              В каком слове верно выделена буква, обозначающая ударный гласный звук?  
        

             В каком варианте ответа выделенное слово употреблено неверно? 
        

              Укажите пример с ошибкой в образовании формы слова.  
        

              Укажите грамматически правильное продолжение предложения. Выбрав пьесу,  
        

              Укажите предложение с грамматической ошибкой (с нарушением синтаксической нормы).  
        

              В каком предложении придаточную часть сложноподчинённого предложения нельзя заменить обособленным определением, выраженным причастным оборотом?  
        

               (1)… (2)Но бывало и по-другому. (3)В XVI веке испанский капитан Себастьян Кабот, плывя вверх по течению реки в Южной Америке, был поражён количеством серебра, которое было у местных индейцев, живших по берегам реки, и решил назвать её Ла-Платой, то есть серебряной  (по-испански «плата»   – MathType@MTEF@5@5@+=feaaguart1ev2aqatCvAUfeBSjuyZL2yd9gz

In [230]:
parse_task(tasks[2])

AssertionError: 

In [86]:
for i,task in enumerate(tasks):
    print(i)
    parse_task(tasks[i])

0
1


AssertionError: 

In [89]:
BeautifulSoup(tasks[1]['html'])

<!DOCTYPE html>
<html test=""><head>
    <title>Нулевая страница</title>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
    <meta charset="utf-8"/>
    <link href="/Content/KimTemplate/kimTmp.css" rel="stylesheet"/>
    <link href="/Content/KimTemplate/kimTmpCheckBox.css" rel="stylesheet"/>
    <script src="/Scripts/es6-shim/es6-shim.min.js"></script>
    <link href="/Content/task-common.css" rel="stylesheet"/>
    <script src="/scripts/task-common.bundle.js"></script>
    <script src="/scripts/task-helper.js"></script>
    <script src="/scripts/shuffleHelper.js"></script>
    <script src="/scripts/fipi-tools.js?rev=2"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=MML_HTMLorMML"></script>
    <script type="text/x-mathjax-config">
        MathJax.Hub.Config({messageStyle: "none"});
    </script>
</head>
<body class="custom-scroll-no inner-answers test noselect">
    <input id="TaskId" type="hidden" value="1569

In [94]:
BeautifulSoup(tasks[0]['html'])

<!DOCTYPE html>
<html test=""><head>
    <title tasktype="RadioButton">Внимательно прочитайте текст задания и выберите верный ответ из списка</title>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
    <meta charset="utf-8"/>
    <link href="/Content/KimTemplate/kimTmp.css" rel="stylesheet"/>
    <link href="/Content/KimTemplate/kimTmpCheckBox.css" rel="stylesheet"/>
    <script src="/Scripts/es6-shim/es6-shim.min.js"></script>
    <link href="/Content/task-common.css" rel="stylesheet"/>
    <script src="/scripts/task-common.bundle.js"></script>
    <script src="/scripts/task-helper.js"></script>
    <script src="/scripts/shuffleHelper.js"></script>
    <script src="/scripts/RadioButtonTest.js"></script>
    <script src="/scripts/fipi-tools.js?rev=2"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=MML_HTMLorMML"></script>
    <script type="text/x-mathjax-config">
        MathJax.Hub.Config({messageStyle: "none"}

In [90]:
BeautifulSoup(tasks[2]['html'])

<!DOCTYPE html>
<html test=""><head>
    <title tasktype="RadioButton">Задание №1. Внимательно прочитайте текст задания и выберите верный ответ из списка</title>
    <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
    <meta charset="utf-8"/>
    <link href="/Content/KimTemplate/kimTmp.css" rel="stylesheet"/>
    <link href="/Content/KimTemplate/kimTmpCheckBox.css" rel="stylesheet"/>
    <script src="/Scripts/es6-shim/es6-shim.min.js"></script>
    <link href="/Content/task-common.css" rel="stylesheet"/>
    <script src="/scripts/task-common.bundle.js"></script>
    <script src="/scripts/task-helper.js"></script>
    <script src="/scripts/shuffleHelper.js"></script>
    <script src="/scripts/RadioButtonTest.js"></script>
    <script src="/scripts/fipi-tools.js?rev=2"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=MML_HTMLorMML"></script>
    <script type="text/x-mathjax-config">
        MathJax.Hub.Config({messageSt